<a href="https://colab.research.google.com/github/RituAnilkumar/EarthEngine/blob/master/Exercise2_multispectral_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"> Google Earth Engine Python API Introductory Tutorial 2 </h1>

**Author:** Ritu Anilkumar


**Last Updated:** 14 January 2020

# Introduction

This exercise is more detailed and will begin a proper Earth Engine Experience. We will access imagery([Sentinel 2](https://sentinel.esa.int/web/sentinel/missions/sentinel-2)), filter it based on our Area of Interest (AOI) and date range of our interest and learn to visualize the multispectral data. Next, we will learn how to perform arithmetic operations and create a spectral indices ([modified normalized difference water index](https://www.tandfonline.com/doi/abs/10.1080/01431160600589179?journalCode=tres20)) and perform some basic logical operations and threshold it to generate a map of water bodies. 

<p align="justify"> The data used in this exercise was captured by the Sentinel 2 sensor bands given by: </p>

><p align="center">Band|<p align="center">Wavelength|<p align="center">Description
>------|------|------
><p align="center">B1|<p align="center">443.9nm|<p align="center">Aerosol
><p align="center">B2|<p align="center">496.6nm|<p align="center">Blue
><p align="center">B3|<p align="center">560nm|<p align="center">Green
><p align="center">B4|<p align="center">664.5nm|<p align="center">Red
><p align="center">B5|<p align="center">	703.9|<p align="center">Red Edge 1
><p align="center">B6|<p align="center">740.2nm|<p align="center">Red Edge 2
><p align="center">B7|<p align="center">782.5nm|<p align="center">Red Edge 3
><p align="center">B8|<p align="center">835.1nm|<p align="center">Near Infra-Red
><p align="center">B8A|<p align="center">864.8nm|<p align="center">Red Edge 4
><p align="center">B9|<p align="center">	945nm|<p align="center">Water vapor
><p align="center">B10|<p align="center">	1373.5nm|<p align="center">Cirrus
><p align="center">B11|<p align="center">1613.7nm|<p align="center">Short Wave Infra-Red 1
><p align="center">B12|<p align="center">2202.4nm|<p align="center">Short Wave Infra-Red 2

Let's explore the dataset using Earth Engine!

# Accessing and filtering Earth Engine Datasets

The first step as always is importing and initilializing Google Earth Engine.

In [1]:
# Import the earth engine library and authenticate with your credentials
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=GAXPNkrqyfuCno7YgayEk73K0B9s7iytjWC3LIZfNwY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/zwHvgMM1SsrvBAUaycOuKKZndequmYkK_YDrt6LD2P2nJbogCTJggWg

Successfully saved authorization token.


Next, we will access the Sentinel 2 dataset. This is stored in 'COPERNICUS/S2'.

In [0]:
# Create a variable containing the location of our area of interest. Longitude,latitude is the syntax followed
loc=ee.Geometry.Point(89.1833,21.9497)

In [11]:
im_col=ee.ImageCollection('COPERNICUS/S2').filterBounds(loc).filterDate(ee.Date('2018-01-01'),ee.Date('2018-12-31'))
num_im=im_col.size()
print(num_im.getInfo())

141


Let's try to filter only those images with cloudy pixels less than 2% of its total area.

In [12]:
im_col=im_col.filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',2).sort('CLOUDY_PIXEL_PERCENTAGE',True)
num_im=im_col.size()
print(num_im.getInfo())

35


Now, let's try to visualize these bands. We saw in the previous exercise about three ways of visualizing. For this, we will stick to thumbnail displays.

In [0]:
im=im_col.first()

In [14]:
from IPython.display import Image
Image(url=im.getThumbUrl({'min': 0, 'max': 4000, 'dimensions': 512, 'bands':['B8','B4','B2']}))

**Task:** Try to modify the cloud cover percentage values and the sorting to True or False. Try to visualize the true color composite.

# Arithmetic and Logical Operations
Next, we will try to create spectral indices. For this study, we will try to generate the modified normalized difference water index. This is given by the formula:
$$mNDWI = \frac{Green-SWIR}{Green+SWIR}$$

For those who generally use NDWI given by
$$mNDWI = \frac{Green-NIR}{Green+NIR}$$
We use the mNDWI which is reported to give better results because it has been oticed that NDWI constantly misclassifies urban pixels as water due to similar normalized difference values. The SWIR absorption of water is also significantly stronger than the NIR characteristics and hence we go with mNDWI

In [0]:
# Function to return the image with a mNDWI band appended
def addmNDWI(image):
  Green=image.select('B3')
  SWIR=image.select('B11')
  mNDWI=image.expression('((B1-B2)/(B1+B2))',{'B1':Green,'B2':SWIR}).rename('mNDWI')
  image=image.addBands(mNDWI)
  return image

In [16]:
# Calling the function
print('Bandnames of the original image are',im.bandNames().getInfo())
mNDWI_im=addmNDWI(im)
print('Bandnames of the image with added mNDWI are',mNDWI_im.bandNames().getInfo())

Bandnames of the original image are ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']
Bandnames of the image with added mNDWI are ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60', 'mNDWI']


Next, let's threshold the mNDWI image and generate a water-not water map

In [17]:
mNDWI=mNDWI_im.select('mNDWI')
mask_im=mNDWI.lte(0.22)
Image(url=mask_im.getThumbUrl({'min': 0, 'max': 1, 'dimensions': 512, 'palette':['black','white']}))

Okay, before we call it a day, for the travel enthusiasts, this is the West Bengal and Bangladesh Sunderbans. By varying the thresholds, you'll see that you will be able to distinguish between the swampy lands and the water. If you haven't been already, definitely visit. The landscape is completely different from the usual natural beauty we are used to in the hills and plains. Take a boat safari. With some luck, you'll be able to see the Royal Bengal Tiger and the Gangetic Dolphin. If not, you'll find plenty of really great birds. So be sure to carry your binoculars and wear not too bright colors!